Shors, by David Finder.

Howdy, welcome to the Quantum demo. We're going to work with _baby_ RSA and _baby_ Shor's algorithm.

First, we're going to cover RSA.

RSA is an asymmetric encryption scheme commonly used for web traffic. Asymmetric means that we can have separate public and private keys. Public key is used for encryption, private key is used for decryption. 

It relies on finding $m^{ed}\equiv m\mod n$ such that e,d, and m are large positive numbers. M is our message, e is our public key, and d is our private key. 

First we need to find our public and private key, and to do this we need to find two very large prime numbers....

In [ ]:
p = 13
q = 23

For some definition of very large. Maybe this is a graph theory class, where 7 is a large number.

In [ ]:
n=p*q

N is our modulus, this is released publically. 
Next, we calculuate $\lambda(n)$, which is the Carmichael Lambda function. 

$\lambda(n) = lcm(\lambda(p),\lambda(q))$.

Given that p,q are prime, $\lambda(p),\lambda(q) = p-1,q-1$

So we take the lcm(12,22), and keep this secret. 

In [ ]:
import math
l = math.lcm(p-1,q-1)
l

Next, we choose an integer e, less than $\lambda(n)$, such that the two are coprime. 

In [ ]:
from sympy import factorint
factorint(132)
e=5

5 looks like a good value for e. 

Next we determine $d\equiv e^{-1}\mod\lambda(n)$


In [ ]:
d=[x for x in range(0,132) if (x*e)%l==1][0]
d

Note: Never make your d this small in an actual implementation. Having a small d makes you vulnerable to [Wiener's attack](https://en.wikipedia.org/wiki/Wiener%27s_attack). Now that we're done snickering:

Our public key is (n,e), and our private key is d. 

We can discard p,q, and $\lambda(n)$

In [ ]:
public = (n,e)
private = d
print(public)
print(private)

So give me a message using ASCII characters:

In [ ]:
message = input("Please give me a plaintext:")

Because our n is so small, we're going to use an ascii encoding, and encrypt each individual value. 

In [ ]:
ascii_values = list(map(lambda x: ord(x),message))

In [ ]:
ascii_values

And now to encrypt, we take each of these integers, and take: $c=m^e \mod(n)$

In [ ]:
cypher_text = list(map(lambda x: (x**e)%n, ascii_values))
print("".join(list(map(lambda x: chr(x),cypher_text))))
cypher_text


beautiful

Now to decrypt, we do the same thing again, with our D private key.

In [ ]:
uncypher_text = list(map(lambda x: (x**d)%n, cypher_text))
print(uncypher_text)
print("".join(list(map(lambda x: chr(x),uncypher_text))))

The magic here stems from the fact that working in addition mod $\lambda (n)$, is equivalent to working in multiplication mod n. 

In particular: $m^{ed} = m^{1+h*\lambda(n)} \mod n$ for some h, because $ed=1 \mod \lambda(n)$ 

We know from Carmichael theorem, that $m^{\lambda(n)}\equiv1$ mod n, which gives us $m^{ed} = m*(e^{\lambda(n)})^h \equiv m*1^h \equiv m$



This is gives us a small problem though.... If we can factor n, which is public, we can find p,q, thus l, and then $\lambda(n)$, which allows us to find d from e by Chinese Remainder theorem. 

Fortunately, factoring numbers is computationally hard. 

The largest we've been able to factor is [RSA-250](https://web.archive.org/web/20200228234716/https://lists.gforge.inria.fr/pipermail/cado-nfs-discuss/2020-February/001166.html)

Which took about 2700 core years for a number with 829 bits. The researchers estimated that a 1024 RSA modulus would take about 500 times as long.[Source](https://eprint.iacr.org/2010/006.pdf). 

These days, 2048 bit modulus are needed. 
Factoring numbers takes sub-exponentially longer with the more bits you add, so with classical computing, we would be in a losing battle against a larger key size. 

Enter quantum computing and Shors algorithm. 


So this is where my headaches began. 

The native implementation of Shor's algorithm on Amazon only factors 15 properly. It also says it factors 21 and 35 properly, but does so improperly.

I tried for a while to get it done using PennyLane. Pennylane has a cool feature where it can implement arbitrary modular multiplication [using this](https://arxiv.org/abs/2112.10537), so I thought to approach it like [this](https://arxiv.org/pdf/1207.0511)




In [ ]:
import pennylane as qml

dev = qml.device("default.qubit")

@qml.qnode(dev)
def circuit():
    wires = qml.registers({"phase": 9, "output": 9, 0: 11, 1: 11, 2: 11, 3: 11, 4: 11, 5: 11, 6: 11, 7: 11, 8: 11})
    qml.BasisState(0, wires["phase"])
    qml.BasisState(1, wires["output"])
    for i in range(0, 9):
        qml.H(i)
    for idx,i in enumerate(map(lambda x: (8**x) % 299, range(1, 10))):
        multi = qml.Multiplier(i, wires["output"], mod=299, work_wires=wires[idx])
        qml.ctrl(multi,wires["phase"][idx],True)
        qml.Barrier(wires[idx])
        #qml.measure(wires[idx],reset=True)
    qml.adjoint(qml.QFT(wires["phase"]))
    return qml.expval(qml.Identity(wires["phase"]+wires["output"]))
#qnode = qml.qnode(circuit,dev)
print(qml.draw(circuit)())                        
output=circuit()
#print(output)

  0: ─╭|Ψ⟩──H─╭●─────────────────────────────────────────────────────────────────────────────
  1: ─├|Ψ⟩──H─│───────────────╭●─────────────────────────────────────────────────────────────
  2: ─├|Ψ⟩──H─│───────────────│───────────────╭●─────────────────────────────────────────────
  3: ─├|Ψ⟩──H─│───────────────│───────────────│───────────────╭●─────────────────────────────
  4: ─├|Ψ⟩──H─│───────────────│───────────────│───────────────│───────────────╭●─────────────
  5: ─├|Ψ⟩──H─│───────────────│───────────────│───────────────│───────────────│──────────────
  6: ─├|Ψ⟩──H─│───────────────│───────────────│───────────────│───────────────│──────────────
  7: ─├|Ψ⟩──H─│───────────────│───────────────│───────────────│───────────────│──────────────
  8: ─╰|Ψ⟩──H─│───────────────│───────────────│───────────────│───────────────│──────────────
  9: ─╭|Ψ⟩────├Multiplier─────├Multiplier─────├Multiplier─────├Multiplier─────├Multiplier────
 10: ─├|Ψ⟩────├Multiplier─────├Multiplier─────├Multiplier───

ValueError: maximum supported dimension for an ndarray is 32, found 117

But this approach requires about 100 qubits, which is far more than most gate based quantum computers can handle, and more than numpy can handle. Unfortunately, we can't clean qubits in Pennylane. Well, we can clean _one_ qubit. This is _so_ helpful.

So we're back to qiskit. I intend to take the Pennylane modulo arithematic libraries and convert them to Qiskit. 


In [1]:
import numpy as np
import qiskit as qs
import qiskit.circuit.library as ql


So this is how we handle addition! We essentially skip the hard part about adding with silly "carry operations" by instead adjusting the phase inside of a QFT.

In [17]:
def add_k_fourier(k):
    add_circuit = qs.QuantumCircuit(qs.QuantumRegister(10,"add"))
    for i in range(9,-1,-1):
        add_circuit.append(ql.PhaseGate(k*np.pi/(2**(9-i))), qargs=[i])
    #print(add_circuit.draw())
    return add_circuit
print(add_k_fourier(299).draw())

       ┌───────────┐
add_0: ┤ P(1.8346) ├
       ├───────────┤
add_1: ┤ P(3.6693) ├
       ├───────────┤
add_2: ┤ P(7.3386) ├
       ├───────────┤
add_3: ┤ P(14.677) ├
       ├───────────┤
add_4: ┤ P(29.354) ├
       ├───────────┤
add_5: ┤ P(58.709) ├
       ├───────────┤
add_6: ┤ P(117.42) ├
       ├───────────┤
add_7: ┤ P(234.83) ├
       ├───────────┤
add_8: ┤ P(469.67) ├
       └┬─────────┬┘
add_9: ─┤ P(299π) ├─
        └─────────┘ 


This is our phase adding system. We carry an extra ancilliary qubit to be able to tell when we get to our highest bit area, so we can invoke the modulus.

tbh, I don't fully understand htis myself. It looks a bit more complicated than it should be, but I guess it's like:
we check if we need to cycle, and then undo that check, and then we keep the mod subtracted if so, and then we finally add k at the end. 



In [18]:

def phase_adder(k,mod): #MAPPING: 0=WORK BIT, 1-10=ACTUAL MATH
    phase_circuit=qs.QuantumCircuit(qs.QuantumRegister(10,"add"),qs.QuantumRegister(1,"adder"))
    phase_circuit.compose(add_k_fourier(k),inplace=True,qubits=range(0,10))
    phase_circuit.compose(add_k_fourier(mod).reverse_ops().inverse(),inplace=True,qubits=range(0,10))
    phase_circuit.append(ql.QFTGate(10).inverse(),qargs=range(0,10))
    phase_circuit.cx(0, 10, ctrl_state=1)
    phase_circuit.append(ql.QFTGate(10),qargs=range(0,10))
    phase_circuit.compose(add_k_fourier(mod).control(1,label="Add Mod"),inplace=True,qubits=[10]+list(range(0,10)))
    phase_circuit.compose(add_k_fourier(k).reverse_ops().inverse(),inplace=True,qubits=range(0,10))
    phase_circuit.append(ql.QFTGate(10).inverse(),qargs=range(0,10))
    phase_circuit.append(ql.XGate().control(1,ctrl_state=0),qargs=[0,10])
    phase_circuit.append(ql.QFTGate(10),qargs=range(0,10))
    phase_circuit.compose(add_k_fourier(k),inplace=True,qubits=range(0,10))
    return phase_circuit
print(phase_adder(8,299).draw())

       ┌─────────┐┌────────────┐┌─────────┐     ┌──────┐┌─────────────────┐»
add_0: ┤ P(π/64) ├┤ P(-1.8346) ├┤0        ├──■──┤0     ├┤0                ├»
       ├─────────┤├────────────┤│         │  │  │      ││                 │»
add_1: ┤ P(π/32) ├┤ P(-3.6693) ├┤1        ├──┼──┤1     ├┤1                ├»
       ├─────────┤├────────────┤│         │  │  │      ││                 │»
add_2: ┤ P(π/16) ├┤ P(-7.3386) ├┤2        ├──┼──┤2     ├┤2                ├»
       └┬────────┤├────────────┤│         │  │  │      ││                 │»
add_3: ─┤ P(π/8) ├┤ P(-14.677) ├┤3        ├──┼──┤3     ├┤3                ├»
        ├────────┤├────────────┤│         │  │  │      ││                 │»
add_4: ─┤ P(π/4) ├┤ P(-29.354) ├┤4        ├──┼──┤4     ├┤4                ├»
        ├────────┤├────────────┤│  qft_dg │  │  │  Qft ││  circuit-159952 │»
add_5: ─┤ P(π/2) ├┤ P(-58.709) ├┤5        ├──┼──┤5     ├┤5                ├»
        └┬──────┬┘├────────────┤│         │  │  │      ││                 │»

We just reduce multiplication to repeated addition in much the same way, by moving into QFT, and then adding $k*2^{i}
$ if the index appears. 

In [19]:
def mul_out_k_mod(k,mod):
    """Performs :math:`x times k` in the registers wires wires_aux"""
    #swap = qs.AncillaRegister(1)
    multiply_out = qs.QuantumCircuit(20)
    multiply_out.append(ql.QFTGate(10),qargs=[19,9,10,11,12,13,14,15,16,17])
    #print(multiply_out.draw())
    for idx in range(0,9):
        codomain = list(range(9,20))
        new_list = [idx]
        new_list.extend(codomain)
        multiply_out.compose(phase_adder(k*(2**(8-idx))%mod,mod).control(1),inplace=True,qubits=new_list)
    multiply_out.append(ql.QFTGate(10).inverse(),qargs=[19,9,10,11,12,13,14,15,16,17])
    return multiply_out
print(mul_out_k_mod(8,299))

                                                                          »
 q_0: ─────────────────■──────────────────────────────────────────────────»
                       │                                                  »
 q_1: ─────────────────┼───────────────────■──────────────────────────────»
                       │                   │                              »
 q_2: ─────────────────┼───────────────────┼───────────────────■──────────»
                       │                   │                   │          »
 q_3: ─────────────────┼───────────────────┼───────────────────┼──────────»
                       │                   │                   │          »
 q_4: ─────────────────┼───────────────────┼───────────────────┼──────────»
                       │                   │                   │          »
 q_5: ─────────────────┼───────────────────┼───────────────────┼──────────»
                       │                   │                   │          »
 q_6: ──────

Mutliplication is multiplying out, and then using swap to get our state back into place. 

In [24]:

def multiplication(k,mod):
    multiply_circuit = qs.QuantumCircuit(qs.QuantumRegister(9,"multi"),qs.QuantumRegister(10,"add"),qs.QuantumRegister(1,"adder"))
    multiply_circuit.compose(mul_out_k_mod(k,mod),inplace=True)
    for x_wire, aux_wire in zip(range(0,9),range(9,19)):
        multiply_circuit.swap(x_wire, aux_wire)
    inv_k = pow(k, -1, mod)
    multiply_circuit.compose(mul_out_k_mod(inv_k,mod).reverse_ops().inverse(),inplace=True)
    return multiply_circuit
print(multiplication(8,299).draw())

                                                                             »
multi_0: ─────────────────■──────────────────────────────────────────────────»
                          │                                                  »
multi_1: ─────────────────┼───────────────────■──────────────────────────────»
                          │                   │                              »
multi_2: ─────────────────┼───────────────────┼───────────────────■──────────»
                          │                   │                   │          »
multi_3: ─────────────────┼───────────────────┼───────────────────┼──────────»
                          │                   │                   │          »
multi_4: ─────────────────┼───────────────────┼───────────────────┼──────────»
                          │                   │                   │          »
multi_5: ─────────────────┼───────────────────┼───────────────────┼──────────»
                          │                   │     

Now, we notice that our $8^n$ iteration step looks very similar to our multi-out step.

Being able to reset at that final range takes us down from pennylane's 116 registers to the 29 registers we wanted to be able to use. 

In [11]:
shors_circuit = qs.QuantumCircuit(qs.QuantumRegister(9,"phase"),qs.QuantumRegister(9,"multi"),qs.AncillaRegister(10,"add"),qs.AncillaRegister(1,"adder"))
shors_circuit.initialize(1,17)
shors_circuit.h(range(0,9))
for i,multiplier in enumerate(list(map(lambda x: ((x*8)%299),range(1,10)))):
    shors_circuit.append(multiplication(multiplier,299).control(1),qargs=[i]+list(range(9,29)))
    shors_circuit.reset(range(18,29))
shors_circuit.append(ql.QFTGate(9).inverse(),range(0,9))
shors_circuit.measure_all()

In [16]:
print(shors_circuit.draw())

       ┌───────────┐
add_0: ┤ P(1.8346) ├
       ├───────────┤
add_1: ┤ P(3.6693) ├
       ├───────────┤
add_2: ┤ P(7.3386) ├
       ├───────────┤
add_3: ┤ P(14.677) ├
       ├───────────┤
add_4: ┤ P(29.354) ├
       ├───────────┤
add_5: ┤ P(58.709) ├
       ├───────────┤
add_6: ┤ P(117.42) ├
       ├───────────┤
add_7: ┤ P(234.83) ├
       ├───────────┤
add_8: ┤ P(469.67) ├
       └┬─────────┬┘
add_9: ─┤ P(299π) ├─
        └─────────┘ 


In [29]:
from qiskit import transpile
trans_circ = transpile(shors_circuit)
print(trans_circ.draw())
print(trans_circ.depth())

               ┌───┐      ┌─────────────────┐                               »
phase_0: ──────┤ H ├──────┤0                ├───────────────────────────────»
               ├───┤      │                 │     ┌───────────────────┐     »
phase_1: ──────┤ H ├──────┤                 ├─────┤0                  ├─────»
               ├───┤      │                 │     │                   │     »
phase_2: ──────┤ H ├──────┤                 ├─────┤                   ├─────»
               ├───┤      │                 │     │                   │     »
phase_3: ──────┤ H ├──────┤                 ├─────┤                   ├─────»
               ├───┤      │                 │     │                   │     »
phase_4: ──────┤ H ├──────┤                 ├─────┤                   ├─────»
               ├───┤      │                 │     │                   │     »
phase_5: ──────┤ H ├──────┤                 ├─────┤                   ├─────»
               ├───┤      │                 │     │             


1. "a" is used to generate a function circuit, which takes x and spits out a^x mod 15. in this case, we represent 8^x mod 15. This is actually a relatively simple task because given x as a binary number, (a=8)^x mod 15. 

This function will apply to a target set of auxillary qubits. 

How is this implemented?
![alt text](./Code.png "Title")

very slowly, it seems!

For each counting qubit, we apply $O(2^i)$ gates. 

hold on, I think I just lost my previous plot. 
I have concluded that this implementation of shors algorithm is bogus. It uses an equal number of auxillary qubits to its normal qubits.


In [35]:
from qiskit_braket_provider import BraketProvider

provider = BraketProvider()

forte = provider.get_backend("TN1")
print(forte.queue_depth())
output = forte.run(trans_circ, shots=1000)

QueueDepthInfo(quantum_tasks={<QueueType.NORMAL: 'Normal'>: '0', <QueueType.PRIORITY: 'Priority'>: '0'}, jobs='0')


NotImplementedError: reset operation not supported by qiskit to braket adapter

"reset operation not supported by qiskit to braket adapter"

OFF TO IBM!

In [ ]:
output.queue_position()


In [ ]:
print(iqm.queue_depth())

In [ ]:
iqm = AwsDevice(Devices.IQM.Garnet)

In [33]:
provider.backends(statuses=["ONLINE"])

[BraketBackend[Aria 1],
 BraketBackend[Forte 1],
 BraketBackend[Garnet],
 BraketBackend[SV1],
 BraketBackend[TN1],
 BraketBackend[dm1]]